In [1]:
import mindpype as mp
import pkgutil, numpy as np

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline.

In [2]:
# Create the pipeline using the factory method
session = mp.Session.create()

Following the creation of the pipeline, we need to create the graphs, which are the representations of the pipeline in MindPype.

Each distinct pipeline would require a different graph object. For this tutorial, we will only require a single graph.

In [3]:
# Create the graph using the factory method
graph = mp.Graph.create(session)

With a graph created, we can now attach our input and output data containers. Since the input data is known, we can create a tensor (the data container in MindPype), using the ```create_from_data()``` factory method. For the output tensor, the library will automatically determine the correct size for the tensor, so we can create a "virtual" tensor, meaning that it can be modified by the library and does not contain any user provided data.       

In [4]:
# Create a random input tensor using the factory method
input_tensor = mp.Tensor.create_from_data(session, shape=(10, 10), data=np.random.rand(10, 10))

# Create an output tensor using the factory method
output_tensor = mp.Tensor.create(session, shape=(10, 10))


With the input and output tensors, we can know add the processing nodes to the pipeline. For this example, we are using a single processing step, or kernel; the baseline correction kernel. We can use the following factory method to add  the kernel to a node, and add the node to the graph we created previously.

Futhermore, we must pass the input and output tensors, as well as the required parameters for the kernel (in this case, the baseline period), to the factory method shown below.

In [5]:
# Create the baseline node, passing the graph, input tensor, output tensor, and baseline period
baseline_node = mp.kernels.BaselineCorrectionKernel.add_baseline_node(graph, input_tensor, output_tensor, baseline_period = [0, 10])

At this point we have created the processing pipeline, and added all of the required components. We can now use a graph method to verify that the structure of the pipeline is valid.

In [6]:
# Verify the structure and inputs/outputs of the graph
graph.verify()


		BaselineCorrectionNode Scheduled


Following the verification of the pipeline, we should now initialize the graph. This step is required for pipelines that have methods that need to be trained or fit (ie. classifiers), but optional for other pipelines.

In [7]:
# Initialize the graph - optional for this pipeline
graph.initialize()

The graph is now ready to execute the provided input data. For other paradigms, we may opt to execute the graph in a loop. If we were using class-separated data, we would need to pass a task label to poll the correct task.

In [8]:
# Execute the graph
graph.execute()



Executing trial with label: None


In [9]:
# Visualize the input and output data
print(input_tensor.data)
print(output_tensor.data)

[[0.96254846 0.77289898 0.98568685 0.0173034  0.15257848 0.07506623
  0.5803459  0.17873377 0.01719537 0.23388644]
 [0.54397459 0.21944208 0.56864143 0.53872194 0.58197083 0.10588558
  0.61097943 0.1329525  0.17725466 0.27356171]
 [0.0030673  0.56898324 0.99580475 0.0483017  0.39269208 0.89395629
  0.70369777 0.65177145 0.66993895 0.44707632]
 [0.03703778 0.71323135 0.51425892 0.02302154 0.66113855 0.70915904
  0.30477382 0.68236756 0.34577397 0.59424369]
 [0.67577325 0.04523109 0.49143855 0.34751092 0.30895924 0.91487255
  0.01987653 0.82764485 0.06902982 0.45552234]
 [0.58225383 0.1171625  0.18152385 0.13378601 0.34832395 0.89696309
  0.59477179 0.04551363 0.45482885 0.68390618]
 [0.77067001 0.67826841 0.80097253 0.46767192 0.16300145 0.43150803
  0.88256771 0.62277941 0.19447759 0.906693  ]
 [0.07073578 0.24876176 0.12985351 0.54553099 0.77667906 0.57449244
  0.29527138 0.36784826 0.1596386  0.91882122]
 [0.30479494 0.56364278 0.40622804 0.6781946  0.81733909 0.073587
  0.15008394 0

In [10]:
# Confirm proper functionality
corrected = input_tensor.data - np.mean(input_tensor.data, axis = -1, keepdims = True)

assert np.allclose(output_tensor.data, corrected)